### Exploration of Taxi Journey Behaviour in New York City Using PyKX

#### Source : Kx Academy

In [4]:
import pykx as kx
import pandas as pd

import numpy as np

from datetime import datetime, date

import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from sklearn.ensemble import RandomForestClassifier

In [7]:
# set the consol size

kx.q.system.display_size = [50,125]

### Read and Preprocessing

In [8]:
# Read the data

taxi = kx.q.read.csv("/home/senthil/data/raw/taxi_data_2016.csv")

taxi.head()

,pickup_date,pickup_time,dropoff_date,dropoff_time,vendor_name,passenger_count,trip_distance,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone,payment_type,total_amount,tip_amount
,,,,,,,,,,,,,,
0,2016.01.01,00:12:22,2016.01.01,00:29:14,"""Creative Mobile""",1h,3.2e,Manhattan,"""Clinton East""","""Manhattan""","""Yorkville East""","""Credit card""",18.36e,3.06e
1,2016.01.01,00:33:43,2016.01.01,00:49:03,"""VeriFone""",1h,6.21e,Queens,"""JFK Airport""","""Queens""","""Richmond Hill""","""Cash""",21.3e,0e
2,2016.01.01,00:46:43,2016.01.01,00:53:12,"""Creative Mobile""",1h,0.4e,Manhattan,"""Clinton East""","""Manhattan""","""Clinton East""","""Cash""",6.8e,0e
3,2016.01.01,00:29:39,2016.01.01,00:44:14,"""Creative Mobile""",1h,0.8e,Manhattan,"""East Village""","""Manhattan""","""Greenwich Village South""","""No charge""",7.3e,0e
4,2016.01.01,00:42:33,2016.01.01,00:58:08,"""VeriFone""",2h,4.65e,Manhattan,"""Clinton East""","""Manhattan""","""East Harlem North""","""Credit card""",21e,4.2e


In [10]:
taxi.shape

(pykx.LongAtom(pykx.q('5245277')), pykx.LongAtom(pykx.q('14')))

In [11]:
taxi.columns

pykx.SymbolVector(pykx.q('`pickup_date`pickup_time`dropoff_date`dropoff_time`vendor_name`passenger_count`trip_distance`pickup_borough`pickup_zone`dr..'))

In [13]:
# Rename some of the column name

taxi = taxi.rename(columns = {
     "vendor_name": "vendor",
        "passenger_count": "passengers",
        "trip_distance": "distance",
        "payment_type": "payment",
        "total_amount": "total",
        "tip_amount": "tip"
})


In [15]:
taxi.head()

,pickup_date,pickup_time,dropoff_date,dropoff_time,vendor,passengers,distance,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone,payment,total,tip
,,,,,,,,,,,,,,
0,2016.01.01,00:12:22,2016.01.01,00:29:14,"""Creative Mobile""",1h,3.2e,Manhattan,"""Clinton East""","""Manhattan""","""Yorkville East""","""Credit card""",18.36e,3.06e
1,2016.01.01,00:33:43,2016.01.01,00:49:03,"""VeriFone""",1h,6.21e,Queens,"""JFK Airport""","""Queens""","""Richmond Hill""","""Cash""",21.3e,0e
2,2016.01.01,00:46:43,2016.01.01,00:53:12,"""Creative Mobile""",1h,0.4e,Manhattan,"""Clinton East""","""Manhattan""","""Clinton East""","""Cash""",6.8e,0e
3,2016.01.01,00:29:39,2016.01.01,00:44:14,"""Creative Mobile""",1h,0.8e,Manhattan,"""East Village""","""Manhattan""","""Greenwich Village South""","""No charge""",7.3e,0e
4,2016.01.01,00:42:33,2016.01.01,00:58:08,"""VeriFone""",2h,4.65e,Manhattan,"""Clinton East""","""Manhattan""","""East Harlem North""","""Credit card""",21e,4.2e


In [32]:
# Consider only data which held on 2016

taxi = taxi.select(where=((kx.Column('pickup_date').year == kx.IntAtom(2016)) & (kx.Column('passengers') != kx.ShortAtom(0))))

In [36]:
# Sort the table based on the "pickup_date", "pickup_time", "dropoff_date", "dropoff_time" columns


taxi = kx.q.xasc(["pickup_date", "pickup_time", "dropoff_date", "dropoff_time"], taxi)


In [38]:
taxi.dtypes

,columns,datatypes
,,
0,pickup_date,"""kx.DateAtom"""
1,pickup_time,"""kx.SecondAtom"""
2,dropoff_date,"""kx.DateAtom"""
3,dropoff_time,"""kx.SecondAtom"""
4,vendor,"""kx.CharVector"""
5,passengers,"""kx.ShortAtom"""
6,distance,"""kx.RealAtom"""
7,pickup_borough,"""kx.SymbolAtom"""
8,pickup_zone,"""kx.CharVector"""


In [41]:
# Type cast the columns to symbol type


taxi = taxi.astype({
    "vendor": kx.SymbolVector,
    "pickup_zone": kx.SymbolVector,
    "dropoff_zone": kx.SymbolVector,
    "payment": kx.SymbolVector
})

taxi.dtypes

,columns,datatypes
,,
0,pickup_date,"""kx.DateAtom"""
1,pickup_time,"""kx.SecondAtom"""
2,dropoff_date,"""kx.DateAtom"""
3,dropoff_time,"""kx.SecondAtom"""
4,vendor,"""kx.SymbolAtom"""
5,passengers,"""kx.ShortAtom"""
6,distance,"""kx.RealAtom"""
7,pickup_borough,"""kx.SymbolAtom"""
8,pickup_zone,"""kx.SymbolAtom"""


In [86]:
#Add bucketing columns

bucket_size_in_min = 30

taxi['pickup_hour'] = kx.q.div(taxi["pickup_time"], 60*bucket_size_in_min)
taxi['pickup_dotw'] = kx.q.mod(taxi["pickup_date"], 7)

## Analyse Taxi Data


The main questions we will try to answer here will revolve around the optimisation of the working days and hours of taxi drivers. We will attempt to answer these question:

    1. What times of the year should drivers take their holidays?
    2. What days of the week should drivers work?
    3. What hours should drivers work?


In [46]:
dates = kx.q.distinct(taxi["pickup_date"])
len(dates)

366

In [47]:
60//30

2

In [87]:
distinct_taxi_hours = kx.q.distinct(taxi["pickup_hour"]) / (60 // bucket_size_in_min)

len(distinct_taxi_hours)

48

In [ ]:
# Do the xgroup based on the date, week, hour columns

grouped_taxi_date = kx.q.xgroup()